In [1]:
import patsy

import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf



# Assignment 1

### 1a
### Data generating process:
The following data generating process represents potential outcomes $Y(1)$, $Y(0)$, and treatment variable $D$, and the observable outcome $Y$.

$
Y(0) = base - X + \epsilon; \sim N(0,1), \epsilon_0 \sim N(0,1)\\
Y(1) = delta + base - X + \epsilon \\
Pr(D=1) = 1  if  X > 2, else Pr(D=1) = 0; Pr(D=0) = 1- Pr(D=1) \\
Y = Y(1)*D + Y(0)*(1-D)\\
$

The process also includes covariates $X$ and a noise term $\epsilon$\

The reason this DGP results in an ATE estimate that does not converge to the two means estimate is because of breakdown of **UNCONFOUNDEDNESS** since the probability of receiving treatment, $D$, is itself a function of the covariates,$X$, i.e., $Pr(D=1) = f(X)$. Thus the outcomes $Y(d)$ are not independent of the treatment itself, $D$. A real life situation that can represent this is treatment/intervention for illnesses, say obesity, where probability of being assigned treatment is dependent on being ill (or over-weight), and not controlling for that, can result is overestimating the true effect of the intervention, via selection bias.


In [2]:
def gen_data(n, d, p, delta, base):
    X = np.random.normal(0, 1, size=(n, d))
    epsilon = np.random.normal(0, 0.1, size=(n,))
    y0 = base - X[:,0] + epsilon
    y1 = delta + base - X[:,0] + epsilon
    D = X[:,0] > p
    y = y1 * D + y0 * (1 - D)
    return y, D, X

## 1b

In [3]:
# Simple two means estimate and calcualtion of variance
def twomeans(y, D):
    hat0 = np.mean(y[D==0]) # mean of outcome of un-treated
    hat1 = np.mean(y[D==1]) # mean of outcome of treated
    V0 = np.var(y[D==0]) / np.mean(1 - D) # asymptotic variance of the mean of outcome of untreated
    V1 = np.var(y[D==1]) / np.mean(D) # asymptotic variance of the mean of outcome of treated
    return hat0, hat1, V0, V1

In [4]:
n = 1000 # n samples
d = 1
delta = 3 # treatment effect
base = 0.3 # baseline outcome
p = 2

y, D, X = gen_data(n, d, p, delta, base) # generate RCT data
hat0, hat1, V0, V1 = twomeans(y, D) # calculate estimation quantities
hat = hat1 - hat0 # estimate of effect
stderr = np.sqrt((V0 + V1) / n)
print(f'Two means estimate of treatment effect is {hat}')
print(f'Confidence interval of treatment effect is {[hat - 1.96 * stderr, hat + 1.96 * stderr]}')

Two means estimate of treatment effect is 0.5436716089434852
Confidence interval of treatment effect is [0.36043918939700403, 0.7269040284899664]


## 1c

In [5]:

# Let's measure coverage: how many times among 100 iterations
# of the experiment, does our 95% confidence interval contain
# the true parameter. It should be 95% of the times
cov, hats, stderrs = [], [], []
for _ in range(100):
    y, D, X = gen_data(n, d, p, delta, base)
    hat0, hat1, V0, V1 = twomeans(y, D)
    hat = hat1 - hat0
    hats.append(hat)
    stderr = np.sqrt((V0 + V1) / n)
    stderrs.append(stderr)
    ci = [hat - 1.96 * stderr, hat + 1.96 * stderr]
    cov += [(ci[0] <= delta) & (delta <= ci[1])] # 1 if CI contains the true parameter
    
print(f'the coverage of the confidence interval is {np.mean(cov)}')
print(f'Bias of the two means estimate: {np.mean(hats) - delta}')
print(f'Standard deviation of the two means estimate: {np.std(hats)}')

the coverage of the confidence interval is 0.0
Bias of the two means estimate: -2.428182349000045
Standard deviation of the two means estimate: 0.07330070900455306









# Assignment 2

## 2a

In [6]:
age_16_17={
    "NV": 58,
    "NU": 61,
    "hit_vaccinated": 0,
    "hit_unvaccinated": 1
}

age_18_64 ={
    "NV": 14443,
    "NU": 14566,
    "hit_vaccinated": 8,
    "hit_unvaccinated": 149
}


def get_ve(x):
    RV = x["hit_vaccinated"]/x["NV"]
    RU = x["hit_unvaccinated"]/x["NU"]
    return RV, RU, (RU-RV)/RU

### Age 16-17


In [7]:
RV, RU, VE = get_ve(age_16_17)
print (f"Overall vaccine efficacy is {VE:.4f}")

Overall vaccine efficacy is 1.0000


### Age 18-64

In [8]:
RV, RU, VE = get_ve(age_18_64)
print (f"Overall vaccine efficacy is {VE:.4f}")

Overall vaccine efficacy is 0.9459


#### Yes, we recover the same vaccine efficiency (100%, 94.6%) for these two age groups as provided by the CDC table

## 2b

In [9]:
np.random.seed(123)
def get_ve_approx_bootstrap(RV, RU, x):
    B = 10000
    effect = 100000 * (RV - RU)
    var_rv = RV * (1 - RV) / x["NV"]
    var_ru = RU * (1 - RU) / x["NU"]
    var_effect = (100000**2) * (var_rv + var_ru)
    RVs = RV  + np.random.normal(0, 1, B) * np.sqrt(var_rv)
    RUs = RU  + np.random.normal(0, 1, B) * np.sqrt(var_ru)
    VEs = (RUs - RVs) / RUs
    CI_VE = np.quantile(VEs, (.025, .975))
    print(f"95% confidence interval of VE is [{CI_VE[0]:.4f}, {CI_VE[1]:.4f}]")


### Age 16-17

In [10]:
RV, RU, _ = get_ve(age_16_17)
get_ve_approx_bootstrap(RV, RU, age_16_17)

95% confidence interval of VE is [1.0000, 1.0000]


### Age 18-64

In [11]:
RV, RU, _ = get_ve(age_18_64)
get_ve_approx_bootstrap(RV, RU, age_18_64)

95% confidence interval of VE is [0.9055, 0.9830]



### steps of approximate bootstrap procedure
- Since the outcomes are binary, i.e., Bernoulli variables, we sample from the 
$p_d + N(0,\frac{V_d}{n})$, $B$ samples, where $p_d$ is the mean outcome for case $d \in \{0,1\}$ and $V_d$ is the variance for case $d$.
- For each sample we calculate the Vaccine Efficacy, $VE$
- The 95% confidence interval is then computed as the range between the [$VE_{\alpha/2$}, VE_{1-\alpha/2}$] percentiles where in our case $\alpha = 0.05$

## 2c

In [12]:
def get_ci_delta_method(x):
    n = x["NU"] + x["NV"]
    hat1, hat0, _ = get_ve(x)
    V0 = hat0 * (1 - hat0) / (x["NU"] / n) # asymptotic variance of baseline mean outcome
    V1 = hat1 * (1 - hat1) / (x["NV"] / n) # asymptotic variance of treatment mean outcome
    hat = 1 - hat1 / hat0
    stderr = np.sqrt( (V0 * hat1**2 / hat0**4 + V1 / hat0**2) / n )
    ci = [hat - 1.96 * stderr, hat + 1.96 * stderr]
    print (f'Vaccine Efficacy is {hat}')
    print (f'Std Err is is {stderr}')
    print (f'CI is {ci}')

In [13]:
get_ci_delta_method(age_16_17)

Vaccine Efficacy is 1.0
Std Err is is 0.0
CI is [1.0, 1.0]


In [14]:
get_ci_delta_method(age_18_64)

Vaccine Efficacy is 0.9458514772489123
Std Err is is 0.01964132842306616
CI is [0.9073544735397026, 0.984348480958122]


The standard error is computed from the $\sqrt{\frac{V_0* (RV)^2}{(RU)^4} + \frac{V_1}{(RU)^2}}$







# Assignment 3

## 3a

In [15]:
data = pd.read_csv("https://raw.githubusercontent.com/VC2015/DMLonGitHub/master/penn_jae.dat", delim_whitespace=True)
n, p = data.shape
data = data[data["tg"].isin([0, 4])]

In [16]:
data["T4"] = np.where(data["tg"] == 4, 1, 0)
data["T4"].value_counts()

0    3354
1    1745
Name: T4, dtype: int64

### Extra controls added
- female * black * agegt54
- exp(dep)

model = "np.log(inuidur1) ~ T4 + (female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2 + (female * black * agegt54) + np.exp(dep)"

In [17]:
model = "np.log(inuidur1) ~ T4 + (female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2 + (female * black * agegt54) + np.exp(dep)"

#### Non-interactive model

In [18]:
cra = smf.ols(model, data)
cra_results = cra.fit(cov_type="HC1")
cra_results.summary()

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 122, but rank is 104
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       np.log(inuidur1)   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     28.60
Date:                Tue, 24 Jan 2023   Prob (F-statistic):               0.00
Time:                        23:14:35   Log-Likelihood:                -8069.9
No. Observations:                5099   AIC:                         1.635e+04
Df Residuals:                    4994   BIC:                         1.704e+04
Df Model:                         104                                         
Covariance Type:                  HC1                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                2.2327      0.195     11.472      0.000       1.851       2.614
C(dep)[T.1]             -0.7479      0.537     -1.392      0.164      -1.801       0.305
C(dep)[T.2]             -0.1458      0.143     -1.020      0.308      -0.426       0.134
T4                      -0.0799      0.036     -2.244      0.025      -0.150      -0.010
female                  -0.1148      0.317     -0.363      0.717      -0.735       0.506
female:C(dep)[T.1]      -0.0283      0.118     -0.241      0.810      -0.259       0.202
female:C(dep)[T.2]       0.1485      0.103      1.443      0.149      -0.053       0.350
black                   -0.3929      0.107     -3.670      0.000      -0.603      -0.183
black:C(dep)[T.1]        0.1534      0.195      0.787      0.431      -0.229       0.535
black:C(dep)[T.2]       -0.1080      0.184     -0.588      0.557      -0.468       0.252
othrace                 -1.1948      0.319     -3.750      0.000      -1.819      -0.570
othrace:C(dep)[T.1]      1.0748      0.642      1.674      0.094      -0.183       2.333
othrace:C(dep)[T.2]      0.0083      0.429      0.019      0.985      -0.833       0.849
q2                       0.2251      0.174      1.292      0.196      -0.116       0.567
C(dep)[T.1]:q2           0.6401      0.573      1.117      0.264      -0.483       1.763
C(dep)[T.2]:q2          -0.0442      0.358     -0.123      0.902      -0.746       0.658
q3                      -0.1555      0.174     -0.895      0.371      -0.496       0.185
C(dep)[T.1]:q3           0.6988      0.574      1.218      0.223      -0.426       1.824
C(dep)[T.2]:q3          -0.0289      0.359     -0.081      0.936      -0.733       0.675
q4                      -0.0487      0.175     -0.278      0.781      -0.392       0.295
C(dep)[T.1]:q4           0.5048      0.574      0.880      0.379      -0.619       1.629
C(dep)[T.2]:q4           0.1393      0.361      0.385      0.700      -0.569       0.848
q5                       0.0392      0.173      0.227      0.820      -0.299       0.377
C(dep)[T.1]:q5           0.5277      0.571      0.924      0.355      -0.591       1.647
C(dep)[T.2]:q5          -0.1697      0.359     -0.472      0.637      -0.874       0.535
q6                      -0.1084      0.219     -0.495      0.621      -0.538       0.321
C(dep)[T.1]:q6           1.0951      0.603      1.817      0.069      -0.086       2.277
C(dep)[T.2]:q6           0.3406      0.388      0.878      0.380      -0.419       1.101
agelt35                 -0.6260      0.308     -2.035      0.042      -1.229      -0.023
C(dep)[T.1]:agelt35      0.0747      0.122      0.613      0.540      -0.164       0.314
C(dep)[T.2]:agelt35      0.0334      0.094      0.354      0.723      -0.152       0.218
agegt54                 -0.3707      0.716     -0.518    

#### Results from new specifications
coef (T4) = -0.0799

std err(T4) = 0.036

#### Results from existing specfications in notebook

coef (T4) = -0.0797	

std err (T4) = 0.036	

#### Interactive model

In [19]:
ira_formula = "0 + (female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2 + (female * black * agegt54) + np.exp(dep)"
X = patsy.dmatrix(ira_formula, data, return_type='dataframe')
X.columns = [f'x{t}' for t in range(X.shape[1])] # clean column names
X = (X - X.mean(axis=0)) # demean all control covariates

# construct interactions of treatment and (de-meaned covariates, 1)
ira_formula = "T4 * ("+ "+".join(X.columns) + ")"
X['T4'] = data['T4']
X = patsy.dmatrix(ira_formula, X, return_type='dataframe')
ira = sm.OLS(np.log(data[["inuidur1"]]), X)
ira_results = ira.fit(cov_type="HC1")
ira_results.summary()

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 245, but rank is 199
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               inuidur1   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     24.44
Date:                Tue, 24 Jan 2023   Prob (F-statistic):               0.00
Time:                        23:14:36   Log-Likelihood:                -8016.1
No. Observations:                5099   AIC:                         1.644e+04
Df Residuals:                    4894   BIC:                         1.778e+04
Df Model:                         204                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.0574      0.021     99.097      0.000       2.017       2.098
T4            -0.0756      0.036     -2.122      0.034      -0.145      -0.006
x0             0.0055      0.285      0.019      0.985      -0.553       0.564
x1            -0.0145      0.395     -0.037      0.971      -0.788       0.759
x2             0.0090      0.111      0.081      0.935      -0.208       0.226
x3            -0.6664      0.409     -1.629      0.103      -1.468       0.136
x4            -0.1736      0.142     -1.220      0.222      -0.452       0.105
x5             0.2167      0.127      1.711      0.087      -0.032       0.465
x6            -0.3937      0.128     -3.087      0.002      -0.644      -0.144
x7             0.1076      0.230      0.468      0.640      -0.343       0.558
x8            -0.2021      0.230     -0.879      0.380      -0.653       0.249
x9            -1.8731      0.598     -3.133      0.002      -3.045      -0.701
x10            2.5199      1.860      1.355      0.175      -1.125       6.165
x11            1.1890      0.575      2.069      0.039       0.063       2.315
x12           -0.0668      0.236     -0.283      0.777      -0.529       0.396
x13            0.2341      0.643      0.364      0.716      -1.027       1.495
x14           -0.2175      0.449     -0.484      0.628      -1.098       0.663
x15           -0.4334      0.236     -1.836      0.066      -0.896       0.029
x16            0.1546      0.645      0.240      0.811      -1.110       1.420
x17           -0.2543      0.451     -0.563      0.573      -1.139       0.630
x18           -0.2922      0.238     -1.230      0.219      -0.758       0.173
x19           -0.0237      0.644     -0.037      0.971      -1.286       1.239
x20           -0.1618      0.452     -0.358      0.721      -1.048       0.725
x21           -0.1841      0.235     -0.783      0.433      -0.645       0.277
x22            0.1395      0.640      0.218      0.827      -1.115       1.394
x23           -0.5627      0.450     -1.250      0.211      -1.445       0.319
x24           -0.4894      0.279     -1.754      0.079      -1.036       0.057
x25            0.4832      0.682      0.708      0.479      -0.854       1.820
x26           -0.0046      0.479     -0.010      0.992      -0.943       0.933
x27           -0.6777      0.410     -1.651      0.099      -1.482       0.127
x28            0.0508      0.150      0.340      0.734      -0.243       0.344
x29            0.1192      0.117      1.023      0.306      -0.109       0.348
x30           -0.3074      0.705     -0.436      0.663      -1.688       1.074
x31            0.0380      0.207      0.184      0.854      -0.368       0.444
x32            0.2195      0.419      0.524      0.600      -0.602       1.041
x33           -0.8383      0.598     -1.402      0.161      -2.010       0.334
x3

#### Results from new specifications
coef (T4) = -0.0756

std err(T4) = 0.036

#### Results from existing specfications in notebook
coef (T4) = -0.0752

std err (T4) = 0.036

## 3b

### Non-robust standard errors with homoskedasticity

In [20]:
cra.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       np.log(inuidur1)   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     3.044
Date:                Tue, 24 Jan 2023   Prob (F-statistic):           2.30e-22
Time:                        23:14:36   Log-Likelihood:                -8069.9
No. Observations:                5099   AIC:                         1.635e+04
Df Residuals:                    4994   BIC:                         1.704e+04
Df Model:                         104                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                2.2327      0.225      9.928      0.000       1.792       2.674
C(dep)[T.1]             -0.7479      0.508     -1.471      0.141      -1.745       0.249
C(dep)[T.2]             -0.1458      0.134     -1.085      0.278      -0.409       0.118
T4                      -0.0799      0.036     -2.241      0.025      -0.150      -0.010
female                  -0.1148      0.347     -0.331      0.741      -0.795       0.565
female:C(dep)[T.1]      -0.0283      0.118     -0.240      0.810      -0.259       0.202
female:C(dep)[T.2]       0.1485      0.103      1.443      0.149      -0.053       0.350
black                   -0.3929      0.097     -4.071      0.000      -0.582      -0.204
black:C(dep)[T.1]        0.1534      0.180      0.851      0.395      -0.200       0.507
black:C(dep)[T.2]       -0.1080      0.160     -0.674      0.500      -0.422       0.206
othrace                 -1.1948      0.449     -2.661      0.008      -2.075      -0.315
othrace:C(dep)[T.1]      1.0748      1.549      0.694      0.488      -1.962       4.111
othrace:C(dep)[T.2]      0.0083      0.533      0.016      0.988      -1.037       1.054
q2                       0.2251      0.203      1.110      0.267      -0.172       0.623
C(dep)[T.1]:q2           0.6401      0.548      1.169      0.242      -0.433       1.714
C(dep)[T.2]:q2          -0.0442      0.415     -0.106      0.915      -0.858       0.770
q3                      -0.1555      0.201     -0.774      0.439      -0.549       0.238
C(dep)[T.1]:q3           0.6988      0.546      1.280      0.201      -0.372       1.769
C(dep)[T.2]:q3          -0.0289      0.415     -0.070      0.944      -0.843       0.785
q4                      -0.0487      0.201     -0.242      0.809      -0.444       0.346
C(dep)[T.1]:q4           0.5048      0.544      0.928      0.354      -0.562       1.572
C(dep)[T.2]:q4           0.1393      0.417      0.334      0.739      -0.679       0.957
q5                       0.0392      0.199      0.197      0.844      -0.350       0.428
C(dep)[T.1]:q5           0.5277      0.542      0.973      0.331      -0.536       1.591
C(dep)[T.2]:q5          -0.1697      0.413     -0.411      0.681      -0.980       0.641
q6                      -0.1084      0.237     -0.458      0.647      -0.572       0.355
C(dep)[T.1]:q6           1.0951      0.578      1.895      0.058      -0.038       2.228
C(dep)[T.2]:q6           0.3406      0.441      0.772      0.440      -0.525       1.206
agelt35                 -0.6260      0.340     -1.839      0.066      -1.293       0.041
C(dep)[T.1]:agelt35      0.0747      0.122      0.614      0.539      -0.164       0.313
C(dep)[T.2]:agelt35      0.0334      0.096      0.350      0.726      -0.154       0.221
agegt54                 -0.3707      0.761     -0.487    

In [21]:
ira.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               inuidur1   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     2.065
Date:                Tue, 24 Jan 2023   Prob (F-statistic):           3.23e-16
Time:                        23:14:36   Log-Likelihood:                -8016.1
No. Observations:                5099   AIC:                         1.644e+04
Df Residuals:                    4894   BIC:                         1.778e+04
Df Model:                         204                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.0574      0.021     98.610      0.000       2.017       2.098
T4            -0.0756      0.036     -2.097      0.036      -0.146      -0.005
x0             0.0055      0.300      0.018      0.985      -0.582       0.593
x1            -0.0145      0.416     -0.035      0.972      -0.830       0.801
x2             0.0090      0.117      0.077      0.939      -0.220       0.238
x3            -0.6664      0.443     -1.504      0.133      -1.535       0.202
x4            -0.1736      0.145     -1.196      0.232      -0.458       0.111
x5             0.2167      0.127      1.701      0.089      -0.033       0.467
x6            -0.3937      0.115     -3.410      0.001      -0.620      -0.167
x7             0.1076      0.219      0.492      0.623      -0.321       0.536
x8            -0.2021      0.202     -1.000      0.317      -0.598       0.194
x9            -1.8731      1.103     -1.698      0.089      -4.035       0.289
x10            2.5199      2.711      0.929      0.353      -2.795       7.835
x11            1.1890      1.465      0.811      0.417      -1.684       4.062
x12           -0.0668      0.276     -0.242      0.809      -0.607       0.473
x13            0.2341      0.682      0.343      0.732      -1.103       1.572
x14           -0.2175      0.492     -0.442      0.659      -1.183       0.748
x15           -0.4334      0.273     -1.586      0.113      -0.969       0.102
x16            0.1546      0.678      0.228      0.820      -1.174       1.483
x17           -0.2543      0.493     -0.516      0.606      -1.220       0.712
x18           -0.2922      0.275     -1.064      0.287      -0.830       0.246
x19           -0.0237      0.676     -0.035      0.972      -1.349       1.302
x20           -0.1618      0.494     -0.327      0.743      -1.131       0.807
x21           -0.1841      0.271     -0.678      0.498      -0.716       0.348
x22            0.1395      0.675      0.207      0.836      -1.183       1.462
x23           -0.5627      0.489     -1.150      0.250      -1.522       0.397
x24           -0.4894      0.307     -1.593      0.111      -1.092       0.113
x25            0.4832      0.711      0.680      0.497      -0.911       1.877
x26           -0.0046      0.521     -0.009      0.993      -1.025       1.016
x27           -0.6777      0.433     -1.566      0.117      -1.526       0.171
x28            0.0508      0.151      0.336      0.737      -0.246       0.348
x29            0.1192      0.118      1.011      0.312      -0.112       0.351
x30           -0.3074      0.812     -0.379      0.705      -1.899       1.284
x31            0.0380      0.214      0.178      0.859      -0.381       0.457
x32            0.2195      0.494      0.444      0.657      -0.749       1.188
x33           -0.8383      0.586     -1.431      0.152      -1.987       0.310
x3

#### The non-robust standard errors are identical to the robust standard errors, i.e., 0.036 in both cases. 
##### This points to the fact that the heteroskedasticity assumption is not necessarily valid for this dataset. The heteroskedasticity assumption is valid when the error on the outcomes is a function of the variance of the variables, which seems not to be the case here.

# Assignment 4

## 4a

In [22]:
### DGP
def dgp(n, p):
    np.random.seed(123)
#     n = 1000             # sample size
    Z = np.random.normal(size=n)         # generate Z
    Y0 = -Z + np.random.normal(size=n)   # conditional average baseline response is -Z
    Y1 = Z + np.random.normal(size=n)    # conditional average treatment effect is +Z
    D = np.random.binomial(1, p, size=n)    # treatment indicator; only 20% get treated 
    Y = Y1 * D + Y0 * (1 - D)  # observed Y
    Z = Z - Z.mean()       # demean Z
    data = pd.DataFrame({"Y": Y, "D": D, "Z": Z})
    return data



In [23]:
data = dgp(1000, 0.2)

CL = smf.ols("Y ~ D", data=data).fit()    
CRA = smf.ols("Y ~ D + Z", data=data).fit()      #classical
IRA = smf.ols("Y ~ D + Z + Z*D", data=data).fit() #interactive approach
# we are interested in the coefficients on variable "D".
print(CL.get_robustcov_results(cov_type="HC1").summary())
print(CRA.get_robustcov_results(cov_type="HC1").summary())
print(IRA.get_robustcov_results(cov_type="HC1").summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.568
Date:                Tue, 24 Jan 2023   Prob (F-statistic):              0.211
Time:                        23:14:36   Log-Likelihood:                -1753.3
No. Observations:                1000   AIC:                             3511.
Df Residuals:                     998   BIC:                             3520.
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0145      0.050      0.290      0.7

Adjusting for covariates (without interactions terms) did **NOT** improve the precision, because the heteroskedasticity robust standard error increased from **0.106 to 0.138**

Adjusting for covariates with interaction terms improved the precision by lowering standard error from **0.106 to 0.080**.

## 4b

### Non-robust standard errors for treatment effect

In [24]:
print(smf.ols("Y ~ D", data).fit().summary())         
print(smf.ols("Y ~ D + Z", data).fit().summary())
print(smf.ols("Y ~ D + Z + Z*D", data).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.381
Date:                Tue, 24 Jan 2023   Prob (F-statistic):              0.240
Time:                        23:14:36   Log-Likelihood:                -1753.3
No. Observations:                1000   AIC:                             3511.
Df Residuals:                     998   BIC:                             3520.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0145      0.049      0.295      0.7

### Non-robust standard errors:
1. "Y ~ D": 0.113
2. "Y ~ D + Z": 0.098
3. "Y ~ D + Z + Z*D": 0.078

This indicates that the homoskedasticity assumption is violated. The calculation of the non-robust standard error assumes homoskedasticity, i.e., error $\epsilon \indep  D, Z$


## 4c

In [25]:
data = dgp(1000, 0.5)


### heteroskedasticity robust standard errors

In [26]:
CL = smf.ols("Y ~ D", data=data).fit()    
CRA = smf.ols("Y ~ D + Z", data=data).fit()      #classical
IRA = smf.ols("Y ~ D + Z + Z*D", data=data).fit() #interactive approach
# we are interested in the coefficients on variable "D".
print(CL.get_robustcov_results(cov_type="HC1").summary())
print(CRA.get_robustcov_results(cov_type="HC1").summary())
print(IRA.get_robustcov_results(cov_type="HC1").summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                  0.005206
Date:                Tue, 24 Jan 2023   Prob (F-statistic):              0.942
Time:                        23:14:36   Log-Likelihood:                -1736.6
No. Observations:                1000   AIC:                             3477.
Df Residuals:                     998   BIC:                             3487.
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0248      0.064     -0.385      0.7

#### Adjusting for covariates (without interaction) did NOT improve precision, since the standard error in both cases was 0.087. However, adding interaction terms improved the precision (std err = 0.061)

### Non robust standard errors

In [27]:
print(smf.ols("Y ~ D", data).fit().summary())         
print(smf.ols("Y ~ D + Z", data).fit().summary())
print(smf.ols("Y ~ D + Z + Z*D", data).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                  0.005212
Date:                Tue, 24 Jan 2023   Prob (F-statistic):              0.942
Time:                        23:14:36   Log-Likelihood:                -1736.6
No. Observations:                1000   AIC:                             3477.
Df Residuals:                     998   BIC:                             3487.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0248      0.062     -0.402      0.6

#### The non-robust errors are the same as the robust standard errors are the same for the cases without interactions, and with interactions, respectively.

### 4d

$$ 

In [28]:
V_1 <= V2 E[D] = 0.5, D INDPENDENT OF W
SAME CONCLUSION AS SLIDE, HOW DO WE GET TO THE RESULT.

SyntaxError: invalid syntax (1619646460.py, line 1)